# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 02**: Dynamic Function Execution Using a Dictionary

**Fecha**: 04/02/2024

**Nombre del Estudiante**: Arturo Benjamin Vergara Romo

**Profesor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab03") \
    .master("spark://14ec38a9be73:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/14 01:28:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [44]:
posts = ["Learning #BigData with #PySpark is fun! #AI",
         "#AI is transforming the world. #BigData #MachineLearning",
         "I love #PySpark and #BigData. #AI #DataScience",
         "#DataScience and #AI are the future. #BigData",
         "#PySpark is awesome! #BigData #AI"]

In [45]:
post_rdd = sc.parallelize(posts)

In [46]:
def extract_hasthags(post):
    return  [word for word in post.split() if word.startswith("#")]

hashtags_rdd = post_rdd.flatMap(extract_hasthags)

In [47]:
hashtags_rdd.collect()

['#BigData',
 '#PySpark',
 '#AI',
 '#AI',
 '#BigData',
 '#MachineLearning',
 '#PySpark',
 '#BigData.',
 '#AI',
 '#DataScience',
 '#DataScience',
 '#AI',
 '#BigData',
 '#PySpark',
 '#BigData',
 '#AI']

In [48]:
hashtags_pairs_rdd = hashtags_rdd.map(lambda x : (x,1))
hashtags_pairs_rdd.collect()

[('#BigData', 1),
 ('#PySpark', 1),
 ('#AI', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#MachineLearning', 1),
 ('#PySpark', 1),
 ('#BigData.', 1),
 ('#AI', 1),
 ('#DataScience', 1),
 ('#DataScience', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#PySpark', 1),
 ('#BigData', 1),
 ('#AI', 1)]

#### Contar las ocurrencias de hashtag con countByValue y reduceByKey

In [49]:
hashtags_count = hashtags_rdd.countByValue()
hashtags_count

defaultdict(int,
            {'#BigData': 4,
             '#PySpark': 3,
             '#AI': 5,
             '#MachineLearning': 1,
             '#BigData.': 1,
             '#DataScience': 2})

In [50]:
reduced_hashtags_rdd = hashtags_pairs_rdd.reduceByKey(lambda a, b: a + b)
reduced_hashtags_rdd.collect()

[('#BigData', 4),
 ('#AI', 5),
 ('#BigData.', 1),
 ('#DataScience', 2),
 ('#PySpark', 3),
 ('#MachineLearning', 1)]

#### Agrupar los hashtags por longitud

In [51]:
hashtags_by_length = hashtags_rdd.map(lambda hashtag: (len(hashtag), hashtag))
hashtags_by_length.collect()

[(8, '#BigData'),
 (8, '#PySpark'),
 (3, '#AI'),
 (3, '#AI'),
 (8, '#BigData'),
 (16, '#MachineLearning'),
 (8, '#PySpark'),
 (9, '#BigData.'),
 (3, '#AI'),
 (12, '#DataScience'),
 (12, '#DataScience'),
 (3, '#AI'),
 (8, '#BigData'),
 (8, '#PySpark'),
 (8, '#BigData'),
 (3, '#AI')]

##### Opción 1 (No distingue por hashtag si dos tienen la misma longitud)

In [52]:
grouped_hashtags = hashtags_by_length.groupByKey()
print("Hashtags Grouped by Length:")
for length, hashtags in grouped_hashtags.collect():
    print(f"Length {length}: {list(hashtags)}")

Hashtags Grouped by Length:
Length 8: ['#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData']
Length 16: ['#MachineLearning']
Length 12: ['#DataScience', '#DataScience']
Length 3: ['#AI', '#AI', '#AI', '#AI', '#AI']
Length 9: ['#BigData.']


##### Opción 2 (Hace una combinación del hashtag con su longitud para la key y mapea solamente la información necesaria)

In [54]:
hashtags_pairs_rdd = hashtags_rdd.map(lambda x: ((len(x), x), x))

grouped_hashtags = hashtags_pairs_rdd.groupByKey()

formatted_rdd = grouped_hashtags.map(lambda x: (x[0][0], list(x[1])))

print("Hashtags Grouped by Length:")
for length, tags in formatted_rdd.collect():
    print(f"Length {length}: {tags}")

Hashtags Grouped by Length:
Length 8: ['#BigData', '#BigData', '#BigData', '#BigData']
Length 12: ['#DataScience', '#DataScience']
Length 8: ['#PySpark', '#PySpark', '#PySpark']
Length 3: ['#AI', '#AI', '#AI', '#AI', '#AI']
Length 16: ['#MachineLearning']
Length 9: ['#BigData.']
